## Mapping human gait using unsupervised AI
This notebook demonstrates how to use the `gait_mapper` package to train a variational autoencoder for mapping human gait.

In [5]:
import numpy as np
import gait_mapper
import gait_mapper.vae
from pathlib import Path

## 1 - Create the variational autoencoder
Call the `vae` module from `gait_mapper` and create the encoder and decoder network.

In [2]:
# define parameters for the model
window_length = 200
degree_of_freedom = 6
latent_features = 6

vae = gait_mapper.vae.VAE(window_length, degree_of_freedom, latent_features)

2022-04-23 13:08:51.721997: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-23 13:08:51.722024: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-23 13:08:51.722034: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DrYang): /proc/driver/nvidia/version does not exist
2022-04-23 13:08:51.722159: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# overview of encoder network
vae.encoder.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 6)]     0                                            
__________________________________________________________________________________________________
conv1 (Conv1D)                  (None, 196, 64)      1984        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 98, 64)       0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv1D)                  (None, 96, 64)       12352       max_pooling1d[0][0]              
______________________________________________________________________________________________

In [4]:
# overview of decoder network
vae.decoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 6)]               0         
_________________________________________________________________
dense (Dense)                (None, 64)                448       
_________________________________________________________________
reshape (Reshape)            (None, 1, 64)             0         
_________________________________________________________________
conv1d_transpose (Conv1DTran (None, 3, 16)             3088      
_________________________________________________________________
conv1d_transpose_1 (Conv1DTr (None, 7, 32)             2592      
_________________________________________________________________
up_sampling1d (UpSampling1D) (None, 35, 32)            0         
_________________________________________________________________
conv1d_transpose_2 (Conv1DTr (None, 39, 64)            1030

## 2 - Loading the dataset

In [13]:
# get the test data
repo_path = Path().resolve().parent
data_folder = Path(repo_path, 'tests/test_data')
# prepare data for the network
raw_data = np.load(Path(data_folder,
 "stored_groupsplit_withoutS01722_latentfeatures_2_frequency_20.npy"))
data = raw_data[:4800].reshape(-1, window_length, degree_of_freedom)

## 3 - Train model

In [15]:
# choose the optimizer
vae.compile(optimizer="adam")
# train the model
history = vae.fit(data, epochs=10, batch_size=2)

Epoch 1/10
2/2 [==============================] - 1s 4ms/step - loss: 2439.3441 - reconstruction_loss: 1585.9705 - kl_loss: 729.5651
Epoch 2/10
2/2 [==============================] - 0s 3ms/step - loss: 2164.5172 - reconstruction_loss: 1394.0663 - kl_loss: 516.2867
Epoch 3/10
2/2 [==============================] - 0s 3ms/step - loss: 1723.2941 - reconstruction_loss: 1458.6663 - kl_loss: 379.1301
Epoch 4/10
2/2 [==============================] - 0s 4ms/step - loss: 1730.0779 - reconstruction_loss: 1423.9127 - kl_loss: 116.9447
Epoch 5/10
2/2 [==============================] - 0s 3ms/step - loss: 1596.9126 - reconstruction_loss: 1292.0602 - kl_loss: 127.4485
Epoch 6/10
2/2 [==============================] - 0s 3ms/step - loss: 921.2072 - reconstruction_loss: 1043.2390 - kl_loss: 156.3671
Epoch 7/10
2/2 [==============================] - 0s 3ms/step - loss: 756.0377 - reconstruction_loss: 792.5471 - kl_loss: 186.9400
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 90

In [17]:
history.history['loss']

[2315.53564453125,
 1910.35302734375,
 1837.79638671875,
 1540.857421875,
 1419.5087890625,
 1199.606201171875,
 979.487060546875,
 790.7210693359375,
 607.0169067382812,
 641.4805908203125]